In [1]:
# Python
import pandas as pd
import numpy as np
from fbprophet import Prophet
%matplotlib inline

In [2]:
# Python
df = pd.read_csv('./data/example_wp_peyton_manning.csv')
df.head()

,ds,y
0,2007-12-10,14629
1,2007-12-11,5012
2,2007-12-12,3582
3,2007-12-13,3205
4,2007-12-14,2680


In [3]:
# Python
def forecast(tdf):
  forecast_df = tdf[['ds', 'y']]
  
  m = Prophet()
  m.fit(forecast_df)
  
  future = m.make_future_dataframe(periods=365)
  forecast = m.predict(future)
  
  m.plot(forecast)
  
  return m, forecast

In [4]:
m, forecast = forecast(df)

In [5]:
# Python
m.plot_components(forecast);


In [6]:
def save_to_mlflow(tdf, m, forecast):
  import mlflow
  from datetime import date
  
  filename = '/dbfs/FileStore/clemens/' + tdf['store_id'].iloc[0] + '.png'
  print (filename)
  m.plot(forecast).savefig(filename)
    
  with mlflow.start_run():    
    mlflow.log_param('forecast_date', date.today())
    mlflow.log_param('store_id', tdf['store_id'].iloc[0])
    mlflow.log_artifact(filename)

INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0